## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,6.69,100,98,1.14,Overcast clouds
1,1,Tocopilla,CL,-22.0920,-70.1979,72.81,70,19,12.06,Few clouds
2,2,Avarua,CK,-21.2078,-159.7750,80.65,83,75,16.11,Broken clouds
3,3,Pangai,TO,-19.8000,-174.3500,81.50,79,91,20.76,Light rain
4,4,Ankazoabo,MG,-22.2833,44.5167,76.14,60,0,5.23,Clear sky


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [19]:
# 4a. Determine if there are any empty rows.
city_data_df.count()

City_ID                200
City                   200
Country                199
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df = city_data_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.65,Broken clouds,-21.2078,-159.7750,
3,Pangai,TO,81.50,Light rain,-19.8000,-174.3500,
4,Ankazoabo,MG,76.14,Clear sky,-22.2833,44.5167,
5,Port Blair,IN,75.18,Clear sky,11.6667,92.7500,
7,Namatanai,PG,78.64,Overcast clouds,-3.6667,152.4333,
10,Busselton,AU,79.05,Broken clouds,-33.6500,115.3333,
12,Mahebourg,MU,80.26,Moderate rain,-20.4081,57.7000,
14,Atuona,PF,78.31,Clear sky,-9.8000,-139.0333,
22,Cidreira,BR,80.40,Broken clouds,-30.1811,-50.2056,
25,Saint-Pierre,RE,80.24,Heavy intensity rain,-21.3393,55.4781,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{row['Lat']},{row['Lng']}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("No hotel found... skipping")
        

No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping
No hotel found... skipping


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))